In [1]:
%matplotlib inline

import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import plotly
import plotly.express as px
import plotly.io as pio
import plotly.offline as pyo
import seaborn as sns
import plotly.graph_objects as go
from sklearn.decomposition import PCA
import sys
import pathlib
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler

sys.path.insert(1, './fma_metadata/')
from fma_metadata import utils

import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (13, 5)

In [2]:
# loading datasets
echonest = utils.load('/Users/gaetanoantonicchio/Desktop/dataset_DM2/fma_metadata/echonest.csv')
tracks = utils.load('/Users/gaetanoantonicchio/Desktop/dataset_DM2/fma_metadata/tracks.csv')

In [14]:
rock_genre = tracks['track'][['genre_top']][tracks['track']['genre_top'] == 'Rock']
echonest_fatures = echonest['echonest','audio_features'].merge(rock_genre,on='track_id')

,genre_top
track_id,
135,Rock
136,Rock
151,Rock
152,Rock
153,Rock
...,...
155315,Rock
155316,Rock
155317,Rock
